In [2]:
import dash
import pandas as pd
import sqlite3
import plotly.express as px
from dash import dcc, html
from dash.dependencies import Input, Output


In [3]:
##CREATE DATABASE

df = pd.read_csv("/Users/josh/Documents/github/josh/Datasets/songs_normalize.csv")
connection = sqlite3.connect("/Users/josh/Documents/github/josh/spotify_data.db")
cursor = connection.cursor()
conn = sqlite3.connect('your_database_name.db')


df.to_sql(
    name = 'song_metrics',
    con = connection, 
    if_exists = 'replace', 
    index = False, 
    dtype = { 
        'artist':'text', 
        'song': 'text',
        'duration_ms': 'integer',
        'explicit': 'boolean',
        'year' : 'integer',
        'popularity' :'integer',
        'danceability': 'float',
        'energy' : 'float',
        'key': 'real',
        'loudness': 'float',
        'mode':'real',
        'speechiness': 'float',
        'acousticness':'float',
        'instrumentalness':'float',
        'liveness':'float',
        'valence':'float',
        'tempo':'float',
        'genre':'float'}
)

2000

In [4]:
query = 'SELECT *  FROM song_metrics WHERE genre != "set()"  '

results = pd.read_sql_query(query, connection)
results.head(1000)

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,0,2000,77,0.751,0.834,1.0,-5.444,0.0,0.0437,0.300000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,0,1999,79,0.434,0.897,0.0,-4.918,1.0,0.0488,0.010300,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,0,1999,66,0.529,0.496,7.0,-9.007,1.0,0.0290,0.173000,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,0,2000,78,0.551,0.913,0.0,-4.063,0.0,0.0466,0.026300,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,0,2000,65,0.614,0.928,8.0,-4.806,0.0,0.0516,0.040800,0.001040,0.0845,0.879,172.656,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Katy Perry,California Gurls,234653,0,2012,72,0.791,0.754,0.0,-3.729,1.0,0.0569,0.004460,0.000000,0.1630,0.425,125.014,pop
996,Kesha,Take It Off,215200,0,2010,68,0.729,0.675,5.0,-5.292,0.0,0.0286,0.000041,0.001260,0.0867,0.740,125.036,"pop, Dance/Electronic"
997,INNA,Hot - Play & Win Radio Version,217036,0,2010,42,0.817,0.939,7.0,-6.079,0.0,0.2800,0.079800,0.441000,0.4310,0.599,128.000,pop
998,Far East Movement,Like A G6,216893,0,2010,73,0.435,0.837,3.0,-8.126,1.0,0.4490,0.006760,0.000000,0.1170,0.778,124.913,"hip hop, pop"


In [5]:
results['genre'] = results['genre'].str.split(',').str[0]


In [6]:
grouped = results.groupby(['genre', 'year'])
mean_values = grouped[['popularity']].mean().reset_index()


mean_values.to_json('genre_year.json', orient='records', lines=True)





In [7]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='category-dropdown',
        options=[
            {'label': genre, 'value': genre}
            for genre in mean_values['genre'].unique()
        ],
        value=df['genre'].unique(),  # Set default to the first category
        multi = True,
        placeholder='Select genre'
    ),
    dcc.Graph(id='scatter-plot')
])

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('category-dropdown', 'value')]
)
def update_scatter_plot(selected_categories):
    filtered_df = mean_values[mean_values['genre'].isin(selected_categories)]

    fig = px.scatter(
        filtered_df,
        x='year',
        y='popularity',
        color='genre',
        labels={'year': 'Year', 'popularity': 'Popularity'},
        title='Average Popularity of Genres by Year'
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)